In [2]:
# 📦 Install necessary libraries
!pip install -q transformers datasets peft accelerate

In [3]:
# 🔧 Imports
from transformers import AutoTokenizer, LlamaConfig, LlamaForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
import torch

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
# 📚 Load real dataset (OpenWebText subset for demo)
dataset = load_dataset("openwebtext", split="train[:1%]")  # use 1% for quick demo


README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] yes


urlsf_subset00.tar:   0%|          | 0.00/633M [00:00<?, ?B/s]

urlsf_subset01.tar:   0%|          | 0.00/629M [00:00<?, ?B/s]

...

urlsf_subset17.tar:   0%|          | 0.00/619M [00:00<?, ?B/s]

urlsf_subset18.tar:   0%|          | 0.00/618M [00:00<?, ?B/s]

urlsf_subset19.tar:   0%|          | 0.00/619M [00:00<?, ?B/s]

urlsf_subset20.tar:   0%|          | 0.00/377M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8013769 [00:00<?, ? examples/s]

In [7]:
# 4. Tokenize Dataset
def tokenize(example):
    # Set the padding token to the EOS token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/80138 [00:00<?, ? examples/s]

In [8]:

# 🛠 Define small LLaMA-style model
config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    intermediate_size=2048,
    num_attention_heads=8,
    num_hidden_layers=4,
    max_position_embeddings=512,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

model = LlamaForCausalLM(config)

# 🧩 Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [9]:


# ⚙️ Training setup
training_args = TrainingArguments(
    output_dir="./llama-openwebtext",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=20,
    save_steps=50,
    save_total_limit=1,
    report_to="none"
)



In [ ]:
# 🚀 Train with Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

# 💾 Save the model
model.save_pretrained("./llama-openwebtext")
tokenizer.save_pretrained("./llama-openwebtext")


<ipython-input-10-ea12e1d9481c>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
20,9.996900
40,9.325600
